In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)
sys.path.append(f'{project_path}/ft_events/src/utils')

from google.cloud import bigquery
from google.cloud import storage

import importlib

import numpy as np
import pandas as pd
from plotly import graph_objs as go
import seaborn as sns
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display

import matplotlib.dates as mdates
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import plotly.express as px

from fintrans_toolbox.src import table_utils as t
from fintrans_toolbox.src import bq_utils as bq


client = bigquery.Client()

## Postal area domestic spending

In [ ]:
sql_ret = f"""SELECT time_period_value, cardholder_location, merchant_location, mcg, 
spend, transactions, cardholders
FROM ons-fintrans-data-prod.fintrans_visa.retail_performance_high_streets_towns
WHERE time_period = 'Month' AND 
cardholder_location_level = "POSTAL_AREA" AND
merchant_location_level = "POSTAL_AREA" AND
cardholder_location != 'All' AND
merchant_location != 'All' 
GROUP BY time_period_value, cardholder_location, merchant_location, mcg, spend, transactions, cardholders
ORDER BY time_period_value, cardholder_location, merchant_location, mcg, spend, transactions, cardholders"""

ret_df = client.query(sql_ret).to_dataframe()
ret_df = t.create_date_time(ret_df)

In [ ]:
len(ret_df['mcg'].unique())

In [ ]:
# client = storage.Client()
# bq.boundary_file_download(client, postal_level = "postcode_area", output_location = '')

area_shape = gpd.read_file('postcode_area.shp')

In [ ]:
area_shape.to_crs(epsg=4326).plot(color='lightgrey')


In [ ]:
area_geodata = area_shape.merge(ret_df, how = 'left', left_on = 'postarea', right_on = 'merchant_location')

In [ ]:
area_geodata = area_shape.merge(area_geodata, how = 'left', left_on = 'postarea', right_on = 'cardholder_location')

In [ ]:
area_geodata = area_geodata.rename(columns={'postarea_x': 'card_area', 'geometry_x': 'card_geometry',
                                           'postarea_y': 'merch_area', 'geometry_y': 'merch_geometry',})


In [ ]:
area_geodata

---------------------

### Example: Flows of spend to Cardiff

In [ ]:
cardiff_df = ret_df[(ret_df['merchant_location'] == 'CF') & (ret_df['mcg'] == 'All')].copy()

In [ ]:
# calculating total monthly spend at CF
cardiff_df['merch_sum_spend'] = cardiff_df['spend'].groupby(cardiff_df['date_time']).transform('sum')
# calculating proportion of cardholder spend in CF
cardiff_df['perc_merch_spend'] = (cardiff_df['spend']/cardiff_df['merch_sum_spend'])*100

In [ ]:
cardiff_df[cardiff_df['date_time'] == '2019-01-01']['perc_merch_spend'].sum()

In [ ]:
# merging the shapefile to the df
cardiff_df_geo = area_shape.merge(cardiff_df, how = 'left', left_on = 'postarea', right_on = 'cardholder_location')

In [ ]:
def plot_interactive_map (gdf):
    
    def plot_map(tod):
        fig, ax = plt.subplots(figsize=(10, 10))
        gdf[gdf['time_period_value']==tod].plot(ax=ax, column='perc_merch_spend', legend=True)
        plt.title(f'Cardiff spend {tod}')


        plt.show()

    # create an interactive slider for dates
    tod = list(gdf['time_period_value'].unique())
    tod_slider = widgets.Dropdown(options=tod, description="Select Date:")

    # return the interactive map
    return widgets.interactive(plot_map, tod=tod_slider)

In [ ]:
plot_interactive_map(cardiff_df_geo)

### Too much local/newport/swansea spend. Excluding these to look at non-local flows

In [ ]:
excluded_areas = ['CF', 'SA', 'NP']
non_local_df = cardiff_df[~cardiff_df['cardholder_location'].isin(excluded_areas)].copy()

In [ ]:
non_local_df.sort_values('perc_merch_spend', ascending = False)['cardholder_location'].unique()

In [ ]:
non_local_df_geo = area_shape.merge(non_local_df, how = 'left', left_on = 'postarea', right_on = 'cardholder_location')

#### note: the mapped proportions are with consideration of CF/NP/SA values (e.g. total props != 100 as CF/NP/SA are excluded)

In [ ]:
plot_interactive_map(non_local_df_geo)

In [ ]:
df_max = non_local_df_geo['perc_merch_spend'].max()
df_min = non_local_df_geo['perc_merch_spend'].min()

In [ ]:
def side_by_side_interactive_map(gdf):
    
    def plot_map(tod1, tod2):

        fig, ((ax1,ax2)) = plt.subplots(1,2,sharey = True,figsize = (10,13))

        gdf[gdf['time_period_value']==tod1].plot(ax=ax1, 
                                                 column='perc_merch_spend', 
                                                 legend=True, 
                                                 legend_kwds={'shrink': 0.3}, 
                                                 vmin = df_min, 
                                                 vmax = df_max,
                                                 cmap="Reds")
        
        gdf[gdf['time_period_value']==tod2].plot(ax=ax2, 
                                                 column='perc_merch_spend',
                                                 legend=True, 
                                                 legend_kwds={'shrink': 0.3},
                                                 vmin = df_min, 
                                                 vmax = df_max,
                                                 cmap="Reds")

        ax1.set_title(f'Cardiff spend - {tod1}')
        ax2.set_title(f'Cardiff spend - {tod2}')

        plt.show()

    # create an interactive slider for dates
    tod1 = list(gdf['time_period_value'].unique())
    tod_slider1 = widgets.Dropdown(options=tod1, description="Select Date:")
    
    tod2 = list(gdf['time_period_value'].unique())
    tod_slider2 = widgets.Dropdown(options=tod2, description="Select Date:")

    # return the interactive map
    return widgets.interactive(plot_map, tod1=tod_slider1, tod2 = tod_slider2)

In [ ]:
side_by_side_interactive_map(non_local_df_geo)

In [ ]:
non_local_df_geo[(non_local_df_geo['cardholder_location'] == 'DD') & (non_local_df_geo['date_time'] == '2024-02-01')]

In [ ]:
non_local_df_geo[(non_local_df_geo['cardholder_location'] == 'DD') & (non_local_df_geo['date_time'] == '2024-02-01')]

In [ ]:
non_local_df_geo[(non_local_df_geo['cardholder_location'] == 'DD') & (non_local_df_geo['date_time'] == '2024-02-01')]

### Isolating just Scottish areas

In [ ]:
scottish_areas = ['AB', 'DD', 'DG', 'EH', 'FK', 'G', 'HS', 'IV', 'KA', 'KW', 'KY', 'ML', 'PA', 'PH', 'TD', 'ZE']

In [ ]:
def categorise_areas(area):
    if area in scottish_areas:
        return 'Yes'
    else:
        return 'No'

In [ ]:
non_local_df_tba = non_local_df.copy()
non_local_df_tba['Scotland'] = non_local_df_tba['cardholder_location'].apply(categorise_areas)

In [ ]:
scottish_cf = non_local_df_tba[non_local_df_tba['Scotland'] == 'Yes'].copy()

In [ ]:
scottish_cf = scottish_cf.groupby(['date_time']).agg({"spend" : "sum", "transactions" : "sum", "cardholders" : "sum", "perc_merch_spend" : "sum"}).reset_index()

In [ ]:
def add_nations_periods(fig):

    # define covid lockdown start and end dates
    game_periods = [
        ("2019-02-01", "2019-03-16"),
        ("2020-02-01", "2020-03-08"),
        ("2021-02-06", "2021-03-26"),
        ("2022-02-01", "2022-03-01"),
        ("2023-02-01", "2023-03-01"),
        ("2024-02-01", "2024-03-01"),
    ]
    # change colour of the covid lockdowns
    fillcolor = "grey"

    # add a dummy trace so that we can add the covid periods to the legend
    # add markers to square to make it show as a square in the legend.
    fig.add_trace(
        go.Scatter(
            x=[None],
            y=[None],
            mode="markers",
            marker=dict(color=fillcolor, symbol="square"),
            name="Six Nations",
        )
    )

    # add covid periods to plot
    for start_date, end_date in game_periods:
        fig.add_shape(
            type="rect",
            xref="x",
            yref="paper",
            x0=start_date,
            y0=0,
            x1=end_date,
            y1=1,
            fillcolor=fillcolor,
            opacity=0.5,
            layer="below",
            line_width=0,
        )

    return fig

fig = px.line(
    scottish_cf,
    x="date_time",
    y="perc_merch_spend",
    title=f"Proportion of Scottish spend in Cardiff",
    )
fig = add_nations_periods(fig)
fig.show()